# one-hot encoding

In [9]:
raw_text = """The Little Prince, written by Antoine de Saint-Exupéry, is a poetic tale about a young prince who travels from his home planet to Earth. The story begins with a pilot stranded in the Sahara Desert after his plane crashes. While trying to fix his plane, he meets a mysterious young boy, the Little Prince.

The Little Prince comes from a small asteroid called B-612, where he lives alone with a rose that he loves deeply. He recounts his journey to the pilot, describing his visits to several other planets. Each planet is inhabited by a different character, such as a king, a vain man, a drunkard, a businessman, a geographer, and a fox. Through these encounters, the Prince learns valuable lessons about love, responsibility, and the nature of adult behavior.

On Earth, the Little Prince meets various creatures, including a fox, who teaches him about relationships and the importance of taming, which means building ties with others. The fox's famous line, "You become responsible, forever, for what you have tamed," resonates with the Prince's feelings for his rose.

Ultimately, the Little Prince realizes that the essence of life is often invisible and can only be seen with the heart. After sharing his wisdom with the pilot, he prepares to return to his asteroid and his beloved rose. The story concludes with the pilot reflecting on the lessons learned from the Little Prince and the enduring impact of their friendship.

The narrative is a beautifully simple yet profound exploration of love, loss, and the importance of seeing beyond the surface of things."""

In [10]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

# 문장 토큰
sent_tokens = sent_tokenize(raw_text.lower())

# 불용어
stopword_set = set(stopwords.words('english'))

# 사전 (key=단어, value=빈도)
vocab = {}

# 토큰/정제/정규화
preprocessed_sentences = []

# 소문자 변환, 토큰화, 불용어, 단어길이 2이하 제거
for st in sent_tokens:
    temp = ''
    for wt in word_tokenize(st):
        
        if wt not in stopword_set and len(wt) > 2:
            if wt in vocab:
                vocab[wt] += 1
            else:
                vocab[wt] = 1

            temp += wt + ' '
    
    preprocessed_sentences.append(temp[:-1])


# vocab = sorted(vocab.items(), key=lambda x: x[1], reverse=True)
# print(vocab)
# print(preprocessed_sentences)



In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=15, oov_token='<OOV>')
sequences = tokenizer.texts_to_sequences(preprocessed_sentences)

pad_seqs = pad_sequences(sequences, maxlen=10, truncating='pre')

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

In [ ]:
from tensorflow.keras.utils import to_categorical

one_hot_encodded = to_categorical(padded_seqs)
one_hot_encodded.shape

NameError: name 'padded_seqs' is not defined

In [12]:
texts = ['난 난 은 근 ㅈ매ㅑㅓ ㅁㅈㅇ러 ㅁㅈ어 ㅔ맺 .', '난 은 을 ㅈ먀ㅐㅗ ㅁㅈㅇ, ']

In [14]:
from konlpy.tag import Okt
import re

okt = Okt()

ko_stopwords = ['은', '는', '이', '가', '을', '나']

preprocessed_texts = []

for text in texts:
    tokens = okt.morphs(text, stem=True)
    tokens = [token for token in tokens if token not in ko_stopwords]
    tokens = [token for token in tokens if not re.search(r'[\s.,;:?]', token)]

    preprocessed_texts.append(tokens)

preprocessed_texts

[['난', '난', '근', 'ㅈ', '매', 'ㅑㅓ', 'ㅁㅈㅇ', '러', 'ㅁㅈ', '어', 'ㅔ', '맺다'],
 ['난', 'ㅈ', '먀', 'ㅐㅗ', 'ㅁㅈㅇ']]

In [16]:
# 시퀀스 처리
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(preprocessed_texts)
sequences = tokenizer.texts_to_sequences(preprocessed_texts)
sequences

[[2, 2, 5, 3, 6, 7, 4, 8, 9, 10, 11, 12], [2, 3, 13, 14, 4]]

In [17]:
tokenizer.word_index

{'<OOV>': 1,
 '난': 2,
 'ㅈ': 3,
 'ㅁㅈㅇ': 4,
 '근': 5,
 '매': 6,
 'ㅑㅓ': 7,
 '러': 8,
 'ㅁㅈ': 9,
 '어': 10,
 'ㅔ': 11,
 '맺다': 12,
 '먀': 13,
 'ㅐㅗ': 14}

In [19]:
# 패딩 처리
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded_seqs = pad_sequences(sequences, maxlen=3)
padded_seqs

array([[10, 11, 12],
       [13, 14,  4]], dtype=int32)

In [20]:
# one-hot
from tensorflow.keras.utils import to_categorical

one_hot_encodded = to_categorical(padded_seqs)
one_hot_encodded.shape

(2, 3, 15)

---

In [22]:
from tensorflow.keras import models, layers

input = layers.Input(shape=(3, 16))
x = layers.SimpleRNN(8)(input)
output = layers.Dense(1, activation='sigmoid')(x)

model = models.Model(inputs=input, outputs=output)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 3, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 8)              │           200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 209 (836.00 B)

 Trainable params: 209 (836.00 B)

 Non-trainable params: 0 (0.00 B)

In [24]:
import numpy as np

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

labels = np.array([1, 0, 1])

model.fit(one_hot_encodded, labels, epochs=3)

ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 2
'y' sizes: 3
